In [1]:
#drive.mount('/content/drive')

In [1]:
import seaborn as sns
import numpy as np
from sklearn.metrics import log_loss
import pandas as pd
from scipy.stats import skew,kurtosis,zscore
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [1]:
df = pd.read_csv("../input/machinehack-ml-merchandise-popularity-prediction/MPP_Dataset/Train.csv")

In [1]:
df.head(5)

In [1]:
df['hour'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.hour
df['month'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.month
df['day'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.day
df['year'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.year
df['minute'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.minute

In [1]:
# '''if df.week.all() < 5:

#   df['week'] = 0
# else:
#   df['week'] = 1'''

In [1]:
df.head(2)

In [1]:
df.info()

In [1]:
df.skew()

In [1]:
def distribution_check(df,Cols):
    plt.figure(figsize=(12,7))
    fig = 1
    i = (len(Cols)//3)+1
    for col in Cols:
        sk = " Skewness : " + str(skew(df[col])) +"\nKurtosis :" + str(kurtosis(df[col]))
        plt.subplot(i, 3, fig)
        sns.distplot(df[col]).set_title(sk)
        fig = fig+1

In [1]:
distribution_check(df,['Score_2'])

In [1]:
#df['Score_2'] = np.log(df['Score_2'])

In [1]:
distribution_check(df,['time'])

In [1]:
df.isnull().sum()

In [1]:
df.popularity.nunique()

In [1]:
y = df[['popularity']]
df = df.drop(["popularity","time","Basket_Ratio"],1)

In [1]:
X = df

In [1]:
import seaborn as sns
corr = X.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True)

In [1]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [1]:
#y = np.array(y)

In [1]:
from sklearn import preprocessing
#y = preprocessing.label_binarize(y, classes=[0, 1, 3,4,5])

In [1]:
X = scaler.fit_transform(X,1)
X

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 42)

SMOTE

In [1]:
from imblearn.over_sampling import ADASYN
from collections import Counter

In [1]:
counter = Counter(y_train)

In [1]:
smt = ADASYN()
X_train_sm,y_train_sm = smt.fit_resample(X_train,y_train)

In [1]:
counter = Counter(y_train_sm)

In [1]:
print(counter)

# RF

In [1]:
class_weight=dict({4:60,0:100,3:85,5:90,1:90})

In [1]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 1000,class_weight=class_weight,verbose = 1,n_jobs = -1,random_state = 42)

In [1]:
%%time

clf.fit(X_train, y_train)

pred_new = clf.predict(X_test)

In [1]:
pred_new = clf.predict(X_test)

In [1]:
print ("Accuracy of test set",accuracy_score(y_test, pred_new)*100,"%")

In [1]:
pred_log = clf.predict_proba(X_test)
print ("log loss of test set",log_loss(y_test, pred_log))

In [1]:
0.3362938412561886

Hyper parameter tuning

In [1]:
param_grid = { 
    'n_estimators': [200, 500,300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,10],
    'criterion' :['gini', 'entropy']
}
from sklearn.model_selection import GridSearchCV

In [1]:
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 3)
CV_rfc.fit(X_train, y_train)

In [1]:
CV_rfc.best_params_

# Feature selection

In [1]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)

In [1]:
pred = lr.predict(X_test)
pred1 = lr.predict(X_train)

In [1]:
print ("Accuracy of test set",accuracy_score(y_test, pred)*100,"%")
print ("Accuracy of training set",accuracy_score(y_train, pred1)*100,"%")

Light gbm

In [1]:
import lightgbm as lgb
train = lgb.Dataset(X_train, label=y_train)
test = lgb.Dataset(X_test, label=y_test)

In [1]:
param = {'objective': 'multiclass',
         'num_iterations': 1000,
         'learning_rate': 0.01,  
         'num_leaves': 23,
         'verbose':0,
         'max_depth': 15, 
         'min_data_in_leaf': 25, 
         'max_bin': 10, 
         'min_data_in_bin': 2,   
         'num_class': 6,
         'metric': 'multi_logloss'
         }

In [1]:
lgbm = lgb.train(params=param,
                 train_set=train,
                 num_boost_round=200,
                 valid_sets=[test])

y_pred_class = lgbm.predict(X_test)

In [1]:
predictions = []
for x in y_pred_class:
    predictions.append(np.argmax(x))

print('accuracy:', accuracy_score(y_test, predictions)*100)

# Fitting on Test set

In [1]:
df = pd.read_csv("../input/machinehack-ml-merchandise-popularity-prediction/MPP_Dataset/Test.csv")

In [1]:
#df['time'] = np.log(df['time'])

In [1]:
df['hour'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.hour
df['month'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.month
df['day'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.day
df['year'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.year
#df['minute'] = pd.to_datetime(df['time'].astype(int), unit='s').dt.minute
df = df.drop(["time"],1)

In [1]:
X = df

In [1]:
df

In [1]:
X = scaler.transform(X)
X

In [1]:
pred_log = clf.predict_proba(X)

In [1]:
pred_log

In [1]:
prediction  = pd.DataFrame(pred_log,columns =['0','1','3','4','5'])
prediction

In [1]:
prediction.to_csv('result_MP.csv',index=False)